## Guidance Basics

To be honest, Guidance is a cool project but right now it's not looking promising on maintainability. See [here](https://github.com/guidance-ai/guidance/issues/382). Not sure whats going on with this project. They released a new version on today (14th November 2023) which drastically [updated](https://github.com/guidance-ai/guidance/discussions/429) how the package works, rendering all the example notebooks as useless until they update them. 

- [GitHub Repo](https://github.com/guidance-ai/guidance/tree/main)
- [Docs](https://guidance.readthedocs.io/en/latest/)


In [1]:
import os
import guidance

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

---

### Basic prompt template

In [2]:
prompt = """My favorite flavor is {flavor}"""

# this relies on the environment variable OPENAI_API_KEY being set
llm = guidance.models.OpenAI(model="gpt-4-1106-preview", api_key=os.environ.get('OPENAI_API_KEY'))

llm + prompt

In [3]:
llm = guidance.models.OpenAI(model="gpt-3.5-turbo", api_key=os.environ.get('OPENAI_API_KEY'))

with guidance.system():
    openai_chat = llm + "You are a cat expert."

with guidance.user():
    openai_chat += "What are the smallest cats?"

with guidance.assistant():
    openai_chat += guidance.gen("answer", stop=".")

openai_chat
print(type(openai_chat))


<class 'guidance.models._openai.OpenAIChat'>


In [5]:
llm = guidance.models.OpenAI(model="gpt-3.5-turbo")

openai_chat = llm + 'I like to play with my ' + guidance.gen(stop=' ') + ' in' + guidance.gen(stop=['\n', '.', '!'])

Exception in thread Thread-6 (_start_generator_stream):
Traceback (most recent call last):
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/site-packages/guidance/models/_remote.py", line 99, in _start_generator_stream
    for chunk in generator:
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/site-packages/guidance/models/_openai.py", line 207, in _generator
    raise e
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/site-packages/guidance/models/_openai.py", line 197, in _generator
    generator = self.client.chat.completions.create(
  File "/Users/award40/anaconda3/envs/masterclass/lib/python3.10/site-packages/openai/_utils/_utils.py", line 299, in wrapper
    return fu

KeyboardInterrupt: 

In [ ]:
llm + '19, 18,' + guidance.gen(max_tokens=50)

---

### Lists of options

In [ ]:
# List
people = ['John', 'Mary', 'Bob', 'Alice']

# Dictionaries
ideas = [
    {
        'name': 'truth', 
        'description': 'the state of being the case'
    },
    {
        'name': 'love', 
        'description': 'a strong feeling of affection'
    }
]

# For lists
program1 = guidance('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}''')

program1(people=people)

In [ ]:
# For list of dictionaries
program2 = guidance('''List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

program2(ideas=ideas)

Joining prompts together

In [ ]:
# note that {{>prog_name}} is the same include syntax as in Handlebars
program3 = guidance('''{{>program1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

program3(program1=program1, people=people, ideas=ideas)

In [ ]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = guidance('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''', llm=llm)
program(llm=llm)

In [ ]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}''')
executed_program = program(example='I hate tacos')

In [ ]:
! pip install --upgrade guidance